# 스페이스쉽 타이타닉 : 최적의 오토글루온 학습 경로

AutoGluon을 활용하여 스페이스쉽 타이타닉 경진대회를 위한 머신러닝 파이프라인을 구현합니다. 탐색적 데이터 분석(EDA)을 통해 얻은 특성 엔지니어링을 적용하였으며, 성능 극대화를 위해 AutoGluon의 하이퍼파라미터를 최적화했습니다.

In [1]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor
import os

TRAIN_PATH = "train.csv"
TEST_PATH = "test.csv"
SUBMISSION_PATH = "submission/submission_optimized.csv"
MODEL_PATH = "AutogluonModels/ag-optimized-best-quality"

print("Libraries loaded and paths set.")

Libraries loaded and paths set.


## 1. 데이터 로드

In [2]:
print("Loading data...")
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

# Combine for consistent preprocessing
train_len = len(train_df)
all_data = pd.concat([train_df, test_df], sort=False).reset_index(drop=True)

print(f"Combined shape: {all_data.shape}")

Loading data...
Combined shape: (12970, 14)


## 2. 특성 엔지니어링

In [3]:
# Spending Columns
spending_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
all_data[spending_cols] = all_data[spending_cols].fillna(0)
all_data['TotalSpending'] = all_data[spending_cols].sum(axis=1)
print("Spending features created.")

Spending features created.


### 승객 정보 Features
논리적인 규칙과 지출 습관을 바탕으로 냉동수면(CryoSleep), 나이(Age), VIP 여부, 그리고 목적지(Destination)의 결측치를 추론하여 채웁니다.

In [4]:
# CryoSleep Imputation
all_data.loc[(all_data['CryoSleep'].isna()) & (all_data['TotalSpending'] > 0), 'CryoSleep'] = False
all_data.loc[(all_data['CryoSleep'].isna()) & (all_data['TotalSpending'] == 0), 'CryoSleep'] = True
all_data['CryoSleep'] = all_data['CryoSleep'].astype(bool)

# Age & AgeGroup
all_data['Age'] = all_data['Age'].fillna(all_data['Age'].median())

def update_age_group(age):
    if age <= 4: return 'Baby'
    elif age <= 12: return 'Child'
    elif age <= 19: return 'Teenager'
    elif age <= 40: return 'Adult'
    elif age <= 60: return 'Middle Aged'
    else: return 'Senior'

all_data['AgeGroup'] = all_data['Age'].apply(update_age_group)

# VIP Imputation
all_data.loc[(all_data['VIP'].isna()) & (all_data['TotalSpending'] == 0), 'VIP'] = False
all_data.loc[(all_data['VIP'].isna()) & (all_data['Age'] <= 19), 'VIP'] = False
all_data['VIP'] = all_data['VIP'].fillna(False).astype(bool)

# Destination Imputation
dest_mode = all_data['Destination'].mode()[0]
all_data['Destination'] = all_data['Destination'].fillna(dest_mode)

print("Personal status features engineered.")

Personal status features engineered.


C:\Users\User\AppData\Local\Temp\ipykernel_37092\3917088119.py:22: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  all_data['VIP'] = all_data['VIP'].fillna(False).astype(bool)


### Group and Family Features
그룹 ID와 성씨(Surname)를 추출하여, 가족이나 그룹 내의 관계를 바탕으로 결측치를 정교하게 추론(Imputation)하는 데 활용합니다

In [5]:
# 1. Group & GroupSize
all_data['Group'] = all_data['PassengerId'].str.split('_').str[0]
group_sizes = all_data.groupby('Group').size()
all_data['GroupSize'] = all_data['Group'].map(group_sizes)

# 2. Surname & FamilySize
all_data['Surname'] = all_data['Name'].str.split().str[-1]
# Fill Surname within Group if possible
all_data['Surname'] = all_data.groupby('Group')['Surname'].ffill()
all_data['Surname'] = all_data.groupby('Group')['Surname'].bfill()

# Map FamilySize
family_counts = all_data['Surname'].value_counts()
all_data['FamilySize'] = all_data['Surname'].map(family_counts)
all_data.loc[all_data['Surname'].isna(), 'FamilySize'] = 1 # Default for unknown

# 3. HomePlanet Imputation
all_data['HomePlanet'] = all_data.groupby('Group')['HomePlanet'].ffill()
all_data['HomePlanet'] = all_data.groupby('Group')['HomePlanet'].bfill()
# Map based on Surname if still missing
home_map = all_data.dropna(subset=['HomePlanet']).groupby('Surname')['HomePlanet'].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan)
all_data['HomePlanet'] = all_data['HomePlanet'].fillna(all_data['Surname'].map(home_map))
all_data['HomePlanet'] = all_data['HomePlanet'].fillna(all_data['HomePlanet'].mode()[0])

print("Group and family features engineered.")

Group and family features engineered.


### 객식(Cabin) Features
객실 정보 세분화 및 복원: Cabin 데이터를 Deck(데크), Num(번호), Side(좌우)로 분리하고, 그룹 내 다른 구성원의 정보를 활용하여 누락된 객실 값을 정교하게 채웁니다

In [6]:
# Split Cabin
all_data[['Deck', 'Num', 'Side']] = all_data['Cabin'].str.split('/', expand=True)

# Fill within Group
for col in ['Deck', 'Num', 'Side']:
    all_data[col] = all_data.groupby('Group')[col].ffill()
    all_data[col] = all_data.groupby('Group')[col].bfill()

# Process Num (convert to numeric)
if all_data['Num'].isna().sum() > 0:
    all_data['Num'] = pd.to_numeric(all_data['Num'], errors='coerce')
    all_data['Num'] = all_data['Num'].fillna(all_data['Num'].median())

print("Cabin features engineered.")

Cabin features engineered.


## 3. 데이터 전처리
모델링을 위한 데이터 준비: 과적합(Overfitting)을 유발할 수 있는 고유 식별값 및 중복 열(PassengerId, Name, Cabin, Surname, Group)을 제거하고, 데이터를 다시 학습용(Train)과 테스트용(Test)으로 분리합니다

In [7]:
cols_to_drop = ['PassengerId', 'Name', 'Cabin', 'Surname', 'Group']

train_final = all_data.iloc[:train_len].copy().drop(columns=cols_to_drop)
test_final = all_data.iloc[train_len:].copy().drop(columns=cols_to_drop)

# Ensure target is dropped from test if present
if 'Transported' in test_final.columns:
    test_final = test_final.drop(columns=['Transported'])

# Train target formatting
label = 'Transported'
train_final[label] = train_final[label].astype(bool)

print(f"Final Train Shape: {train_final.shape}")
print(f"Final Test Shape: {test_final.shape}")
print(f"Features: {list(train_final.columns)}")

Final Train Shape: (8693, 18)
Final Test Shape: (4277, 17)
Features: ['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Transported', 'TotalSpending', 'AgeGroup', 'GroupSize', 'FamilySize', 'Deck', 'Num', 'Side']


## 4. AutoGluon 모델 학습
모델 최적화 및 학습: 최상의 앙상블 성능을 위해 best_quality 프리셋을 사용하며, 1시간의 제한 시간, 8겹의 배깅(Bagging), 그리고 2단계 스태킹(Stacking)을 적용합니다. 또한, 과적합을 방지하기 위해 CatBoost 모델의 깊이를 6으로 제한합니다.

CatBoost를 단일 알고리즘으로 사용하되, 8-폴드 배깅(Bagging)과 2단계 스태킹(Stacking) 기술을 적용한 CatBoost 전용 앙상블 모델입니다.<br>best_quality로 1시간 학습 : WeightedEnsemble_L2가 최고 모델로 선정되었으며, 0.8227점이 나왔습니다.

In [ ]:
print("Starting AutoGluon Training...")

hyperparameters = {
    'CAT': {'depth': 6},
}

predictor = TabularPredictor(
    label=label,
    eval_metric='accuracy',
    path=MODEL_PATH,
    problem_type='binary'
).fit(
    train_data=train_final,
    presets='best_quality',
    time_limit=3600, # 1 hour
    num_bag_folds=8,
    num_stack_levels=2,
    hyperparameters=hyperparameters,
)
# 모델별 성적표(리더보드) 출력
print("\n=== [결과 리더보드] ===")
lb = predictor.leaderboard(extra_info=True)
display(lb[['model', 'score_val', 'pred_time_val', 'fit_time']])

# 최고 모델의 이름과 점수 가져오기
# get_model_best() 대신 아래 두 가지 중 하나를 사용합니다.
best_model_name = predictor.model_best 
best_score = lb.loc[lb['model'] == best_model_name, 'score_val'].values[0]

print(f"\n최고 모델: {best_model_name}")
print(f"내부 검증 정확도(Score_Val): {best_score:.4f}")
print("Training Complete.")

Verbosity: 2 (Standard Logging)


Starting AutoGluon Training...


=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.11.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          16
Pytorch Version:    2.9.1+cpu
CUDA Version:       CUDA is not available
Memory Avail:       13.60 GB / 31.72 GB (42.9%)
Disk Space Avail:   251.52 GB / 476.83 GB (52.7%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the


=== [결과 리더보드] ===


,model,score_val,pred_time_val,fit_time
0,LightGBM_BAG_L3,0.840446,17.194179,1631.768937
1,WeightedEnsemble_L4,0.840446,17.197174,1632.407176
2,CatBoost_r177_BAG_L3,0.839756,17.186764,1652.045540
3,CatBoost_BAG_L3,0.839066,17.174221,1701.040648
4,LightGBM_r131_BAG_L3,0.838951,17.237628,1636.552284
...,...,...,...,...
64,NeuralNetTorch_BAG_L1,0.802830,0.353503,102.532231
65,RandomForest_r195_BAG_L1,0.802370,0.525326,2.813130
66,NeuralNetTorch_r79_BAG_L1,0.801795,0.304731,142.996840
67,ExtraTreesGini_BAG_L1,0.801795,0.569327,1.384576



최고 모델: WeightedEnsemble_L4
내부 검증 정확도(Score_Val): 0.8404
Training Complete.


모델을 따로 지정하지 않고, excluded_model_types=[] 설정을 통해 AutoGluon이 모든 모델을 사용해서 데이터의 특성에 맞춰 최적의 모델 조합을 스스로 선택하여 강력한 멀티레이어 스태킹(Multi-layer Stacking) 앙상블을 구축합니다.

In [ ]:
print("Starting AutoGluon Training...")

predictor = TabularPredictor(
    label=label,
    eval_metric='accuracy',
    path='ag_models/spaceship_autogluon_best_quality_v1',
    problem_type='binary'
).fit(
    train_data=train_final,
    presets='best_quality',      # 고품질 앙상블 활성화
    time_limit=3600,             # 1시간 동안 충분히 모델들을 섞음
    num_bag_folds=8,             # 안정적인 점수를 위한 8겹 배깅
    num_stack_levels=2,          # 모델들의 예측치를 다시 학습하는 2단계 스태킹
    excluded_model_types=[], # 모든 모델 사용
)

# 모델별 성적표(리더보드) 출력
print("\n=== [결과 리더보드] ===")
lb = predictor.leaderboard(extra_info=True)
display(lb[['model', 'score_val', 'pred_time_val', 'fit_time']])

# 최고 모델의 이름과 점수 가져오기
# get_model_best() 대신 아래 두 가지 중 하나를 사용합니다.
best_model_name = predictor.model_best 
best_score = lb.loc[lb['model'] == best_model_name, 'score_val'].values[0]

print(f"\n최고 모델: {best_model_name}")
print(f"내부 검증 정확도(Score_Val): {best_score:.4f}")

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.11.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          16
Pytorch Version:    2.9.1+cpu
CUDA Version:       CUDA is not available
Memory Avail:       9.04 GB / 31.72 GB (28.5%)
Disk Space Avail:   250.21 GB / 476.83 GB (52.5%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluo

Starting AutoGluon Training...


Leaderboard on holdout data (DyStack):
                      model  score_holdout  score_val eval_metric  pred_time_test  pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0         LightGBMXT_BAG_L2       0.814700   0.826712    accuracy        6.064746       4.227138  354.107949                 0.202710                0.237131          10.065766            2       True         13
1            XGBoost_BAG_L2       0.814700   0.825547    accuracy        6.180773       4.133321  349.135993                 0.318737                0.143314           5.093811            2       True         21
2   RandomForestGini_BAG_L3       0.811594   0.827747    accuracy        9.165972       7.527830  627.967878                 0.412521                0.514102           1.738129            3       True         27
3     ExtraTreesEntr_BAG_L2       0.810559   0.822053    accuracy        6.059507       4.395024  345.207662     

### 모델 학습 결과 요약 및 성능 평가

In [12]:
print(predictor.fit_summary())

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val eval_metric  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      CatBoost_BAG_L1   0.822731    accuracy       0.077698  57.885597                0.077698          57.885597            1       True          1
1  WeightedEnsemble_L2   0.822731    accuracy       0.080682  57.904061                0.002984           0.018465            2       True          2
Number of models trained: 2
Types of models trained:
{'StackerEnsembleModel_CatBoost', 'WeightedEnsembleModel'}
Bagging used: True  (with 8 folds)
Multi-layer stack-ensembling used: False 
Feature Metadata (Processed):
(raw dtype, special dtypes):
('category', [])  : 5 | ['HomePlanet', 'Destination', 'AgeGroup', 'Deck', 'Side']
('float', [])     : 9 | ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', ...]
('int', [])       : 1 | ['GroupSize']
('int', ['bool']) : 2 | ['CryoSl

c:\Users\User\miniconda3\envs\DS\Lib\site-packages\autogluon\core\utils\plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


## 5. 예측 및 제출 파일 생성
학습된 모델을 사용하여 최종 예측을 수행하고, 캐글 제출 양식에 맞는 submission.csv 파일을 생성합니다

WeightedEnsemble_L2 : 0.8227점, 케글 점수 : 0.81038점

In [9]:
print("Generating predictions...")
# 예측 수행
y_pred = predictor.predict(test_final)

# 제출용 데이터프레임 생성 (인덱스 꼬임 방지를 위해 .values 사용)
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'].values, 
    'Transported': y_pred.values 
})

# 데이터 타입 보정 (True/False 형태인지 확인)
submission['Transported'] = submission['Transported'].astype(bool)

# 저장 폴더 확인 및 저장
if not os.path.exists('submission'):
    os.makedirs('submission')

submission.to_csv(SUBMISSION_PATH, index=False)

print(f"Submission saved successfully to {SUBMISSION_PATH}")
print("\n=== [제출 파일 상단 5행 확인] ===")
print(submission.head())

Generating predictions...
Submission saved successfully to submission/submission_optimized.csv

=== [제출 파일 상단 5행 확인] ===
  PassengerId  Transported
0     0013_01         True
1     0018_01        False
2     0019_01         True
3     0021_01         True
4     0023_01        False
